In [ ]:
# "제목 !!"
> "요약!! "

- toc:true- branch: master
- badges: true
- comments: true
- author: In Chan
- categories: [jupyter, deep learning]

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.init
import torch.optim as optim
import numpy as np
from scipy import io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
# import Engine
from utils import *
import utils
import optuna
import math
from torch.autograd import Function
import os
from torch.utils.tensorboard import SummaryWriter
from model import Classifier_Tunning, Extractor, Classifier, Extractor_Tunning
from Engine import Engine
import time

In [2]:

def objective(trial):
    # log, logclose = create_logger(log_filename=log_filename)
    
    Activation_name_generator = trial.suggest_categorical("Activate Function for Generator", ["ReLU", "LeakyReLU", "ELU","Tanh"])
    Activation_generator = getattr(nn, Activation_name_generator)
    Feature_Extractor = Extractor_Tunning(Activation_generator)
    
    Activation_name_classifier = trial.suggest_categorical("Activate Function for Classifier", ["ReLU", "LeakyReLU", "ELU","Tanh"])
    Activation_classifier = getattr(nn, Activation_name_classifier)
    Classifier1 = Classifier_Tunning(Activation_classifier)
    Classifier2 = Classifier_Tunning(Activation_classifier)

    #############################################################
    DEVICE = 'cuda'
    EPOCH = 30 

    Feature_Extractor = Feature_Extractor.to(DEVICE)
    Classifier1 = Classifier1.to(DEVICE)
    Classifier2 = Classifier2.to(DEVICE)

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "Adagrad","Adadelta"])

    lr_E = trial.suggest_float("lr", 1e-7, 1e-2, log=True)
    lr_C = trial.suggest_float("lr", 1e-7, 1e-2, log=True)
    
    optimizer_E = getattr(optim, optimizer_name)(Feature_Extractor.parameters(), lr=lr_E)
    optimizer_C1 = getattr(optim, optimizer_name)(Classifier1.parameters(), lr=lr_C)
    optimizer_C2 = getattr(optim, optimizer_name)(Classifier2.parameters(), lr=lr_C)
    
    # Criterion = nn.CrossEntropyLoss()
    
    scheduler_name = trial.suggest_categorical("scheduler", ["LambdaLR", "StepLR", "CosineAnnealingLR",'None'])
    if scheduler_name == "LambdaLR" :
        scheduler_E = optim.lr_scheduler.LambdaLR(optimizer = optimizer_E,lr_lambda = lambda epoch:0.95 **epoch)
        scheduler_C1 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C1,lr_lambda = lambda epoch:0.95 **epoch)
        scheduler_C2 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C2,lr_lambda = lambda epoch:0.95 **epoch)
    if scheduler_name == "StepLR" :
        scheduler_E = optim.lr_scheduler.StepLR(optimizer = optimizer_E,step_size = 6 ,gamma=0.75)
        scheduler_C1 = optim.lr_scheduler.StepLR(optimizer = optimizer_C1,step_size = 6 ,gamma=0.75)
        scheduler_C2 = optim.lr_scheduler.StepLR(optimizer = optimizer_C2,step_size = 6 ,gamma=0.75)
    if scheduler_name == "CosineAnnealingLR" :
        scheduler_E = optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer_E,T_max=5)
        scheduler_C1 = optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer_C1,T_max=5)
        scheduler_C2 = optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer_C2,T_max=5)
    if scheduler_name == "None" :
        scheduler_E = optim.lr_scheduler.LambdaLR(optimizer = optimizer_E,lr_lambda = lambda epoch:1 **epoch)
        scheduler_C1 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C1,lr_lambda = lambda epoch:1 **epoch)
        scheduler_C2 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C2,lr_lambda = lambda epoch:1 **epoch)
        
    N = trial.suggest_int("Training Number of Generator", 1, 10) # Training Number of Generator 
    #############################################################

    # Best_Loss = np.inf
    Best_Accuracy = 0

    log, logclose = create_logger(log_filename=log_filename,display=False)
    log(f'-------------------------------------------- Trial Number = {trial.number} ---------------------------------------------')
    log('#################################################################################')
    log('############################### Hyper Prameters #################################')
    log(f'Generator Activation : {Activation_name_generator}')
    log(f'Generator Learning Rate = {lr_E}')
    log(f'Classifier Activation : {Activation_name_classifier}')
    log(f'Classifier Learning Rate = {lr_C}')
    log(f'Optimizer Name : {optimizer_name}')
    log(f'Scheduler Name : {scheduler_name}')
    log(f'Training Number of Generator : {N}')
    log('############################### Hyper Prameters #################################')
    log('#################################################################################')
    log('\n')
    # print('Start Learning')
    Eng = Engine(optimizer_E,optimizer_C1,optimizer_C2,Feature_Extractor,Classifier1,Classifier2,DEVICE)
    for epoch in range(EPOCH) : 
        Loss_A , Loss_B, Loss_C = Eng.train(source_loader, target_loader,N)
        ACC1 , ACC2, Test_Loss_1, Test_Loss_2 = Eng.test(test_loader)
        
        scheduler_E.step()
        scheduler_C1.step()
        scheduler_C2.step()

        log, logclose = create_logger(log_filename=log_filename)
        log(f'###############Epoch = {epoch}###############')
        log(f'Loss A = {Loss_A:0.4f} , Loss B = {Loss_B:0.4f} , Loss C = {Loss_C:0.4f}')
        log(f'Accuracy 1 = {ACC1*100:0.2f}, Accuracy 2 = {ACC2*100:0.2f} ')
        log(f'Test Loss 1 = {Test_Loss_1:0.4f}, Test Loss 2 = {Test_Loss_2:0.4f} ')

        if max(ACC1,ACC2) > Best_Accuracy : 
            Best_Accuracy = max(ACC1,ACC2)
            log(f'--------------------------------Best Accuracy = {Best_Accuracy * 100}--------------------------------')
        
        logclose()
        trial.report(Best_Accuracy, epoch)
            # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
################################################################
    return Best_Accuracy

In [3]:
name = time.asctime(time.localtime(time.time()))
name = name.replace(' ','_').replace(':','_')
log_dir = f'./log/{name}'
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, 'train.log')

# Data Load
folds = make_folds(10)
S_train_data , S_train_label , S_test_data , S_test_label = source_Load_data(folds)
T_train_data , T_train_label , T_test_data , T_test_label = target_Load_data(folds)
test_data , test_label = test_Load_data(folds)

source_set = utils.CustomDataset(data=S_train_data, label = S_train_label)
source_loader=DataLoader(source_set,batch_size=32,shuffle=True,drop_last=False)

target_set = utils.CustomDataset(data=T_test_data, label = T_test_label)
target_loader = DataLoader(target_set,batch_size=32,shuffle=True,drop_last=False)

test_set = utils.CustomDataset(data=test_data, label = test_label)
test_loader = DataLoader(test_set,batch_size=32,shuffle=True,drop_last=False)

study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=10000)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-07-02 14:45:32,198] A new study created in memory with name: no-name-b892c469-190b-4a28-af38-d03b7acd5182


###############Epoch = 0###############
Loss A = 3.4245 , Loss B = 3.5238 , Loss C = 0.0591
Accuracy 1 = 27.64, Accuracy 2 = 27.49 
Test Loss 1 = 1.5522, Test Loss 2 = 1.6179 
--------------------------------Best Accuracy = 27.641318259385667--------------------------------
###############Epoch = 1###############
Loss A = 2.5375 , Loss B = 2.4093 , Loss C = 0.0503
Accuracy 1 = 27.67, Accuracy 2 = 21.73 
Test Loss 1 = 2.5335, Test Loss 2 = 3.0405 
--------------------------------Best Accuracy = 27.667804323094426--------------------------------
###############Epoch = 2###############
Loss A = 1.7039 , Loss B = 1.4153 , Loss C = 0.0472
Accuracy 1 = 37.14, Accuracy 2 = 37.46 
Test Loss 1 = 3.3043, Test Loss 2 = 2.3709 
--------------------------------Best Accuracy = 37.464448236632535--------------------------------
###############Epoch = 3###############
Loss A = 1.3150 , Loss B = 0.9407 , Loss C = 0.0556
Accuracy 1 = 42.51, Accuracy 2 = 40.21 
Test Loss 1 = 5.1359, Test Loss 2 = 2.9495 

[I 2021-07-02 14:55:58,561] Trial 0 finished with value: 0.5695712457337884 and parameters: {'Activate Function for Generator': 'ReLU', 'Activate Function for Classifier': 'ReLU', 'optimizer': 'RMSprop', 'lr': 0.009503095236086176, 'scheduler': 'CosineAnnealingLR', 'Training Number of Generator': 9}. Best is trial 0 with value: 0.5695712457337884.


###############Epoch = 29###############
Loss A = 0.6204 , Loss B = 0.4825 , Loss C = 0.0366
Accuracy 1 = 53.99, Accuracy 2 = 53.67 
Test Loss 1 = 9.1220, Test Loss 2 = 6.9299 
###############Epoch = 0###############
Loss A = 3.5818 , Loss B = 3.5674 , Loss C = 0.0134
Accuracy 1 = 27.21, Accuracy 2 = 17.00 
Test Loss 1 = 1.7893, Test Loss 2 = 1.7904 
--------------------------------Best Accuracy = 27.20740898748578--------------------------------
###############Epoch = 1###############
Loss A = 3.5644 , Loss B = 3.5518 , Loss C = 0.0115
Accuracy 1 = 26.48, Accuracy 2 = 19.03 
Test Loss 1 = 1.7840, Test Loss 2 = 1.7871 
###############Epoch = 2###############
Loss A = 3.5477 , Loss B = 3.5361 , Loss C = 0.0107
Accuracy 1 = 24.85, Accuracy 2 = 21.07 
Test Loss 1 = 1.7791, Test Loss 2 = 1.7849 
###############Epoch = 3###############
Loss A = 3.5367 , Loss B = 3.5258 , Loss C = 0.0104
Accuracy 1 = 26.47, Accuracy 2 = 21.15 
Test Loss 1 = 1.7770, Test Loss 2 = 1.7835 
###############Epoch 

[I 2021-07-02 15:07:03,941] Trial 1 finished with value: 0.3613730091012515 and parameters: {'Activate Function for Generator': 'ELU', 'Activate Function for Classifier': 'ReLU', 'optimizer': 'Adadelta', 'lr': 0.0016276412174605565, 'scheduler': 'CosineAnnealingLR', 'Training Number of Generator': 10}. Best is trial 0 with value: 0.5695712457337884.


...

###############Epoch = 0###############
Loss A = 1.7753 , Loss B = 1.5944 , Loss C = 0.0291
Accuracy 1 = 57.90, Accuracy 2 = 55.27 
Test Loss 1 = 1.1565, Test Loss 2 = 1.2861 
--------------------------------Best Accuracy = 57.90137940841866--------------------------------


[I 2021-07-03 19:56:28,036] Trial 3559 pruned. 


###############Epoch = 0###############
Loss A = 2.8794 , Loss B = 2.8041 , Loss C = 0.0145
Accuracy 1 = 51.66, Accuracy 2 = 50.33 
Test Loss 1 = 1.4591, Test Loss 2 = 1.5065 
--------------------------------Best Accuracy = 51.65955631399317--------------------------------
###############Epoch = 0###############
Loss A = 2.5316 , Loss B = 2.4034 , Loss C = 0.0186
Accuracy 1 = 66.51, Accuracy 2 = 69.23 
Test Loss 1 = 1.2532, Test Loss 2 = 1.2278 
--------------------------------Best Accuracy = 69.22621587030716--------------------------------


[W 2021-07-03 19:57:06,405] Trial 3560 failed because of the following error: FileNotFoundError(2, 'No such file or directory')
Traceback (most recent call last):
  File "C:\Users\dldls\anaconda3\envs\DL\lib\site-packages\optuna\_optimize.py", line 216, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-2-401ac50e24af>", line 80, in objective
    log, logclose = create_logger(log_filename=log_filename)
  File "C:\Users\dldls\Desktop\210702_Optuna_MCD\utils.py", line 162, in create_logger
    f = open(log_filename, 'a')
FileNotFoundError: [Errno 2] No such file or directory: './log/Fri_Jul__2_14_45_27_2021\\train.log'


FileNotFoundError: [Errno 2] No such file or directory: './log/Fri_Jul__2_14_45_27_2021\\train.log'

In [ ]:
plot_optimization_history(study)
plot_intermediate_values(study)
plot_parallel_coordinate(study)
plot_parallel_coordinate(study, params=["bagging_freq", "bagging_fraction"])
plot_contour(study)
plot_contour(study, params=["bagging_freq", "bagging_fraction"])
plot_slice(study)
plot_slice(study, params=["bagging_freq", "bagging_fraction"])
plot_param_importances(study)
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)
plot_edf(study)
